# Global Food Prices

Understanding how xxxxx



___________________________________________
Author: Andreia Ferreira Almeida

In [1]:
# Imports
import os
import pandas as pd
import numpy as np


import warnings
warnings.filterwarnings('ignore')

### Loading data

In [2]:
proj_dir = os.path.dirname(os.path.abspath("wfp_market_food_prices.csv"))
data = os.path.join(proj_dir, "wfp_market_food_prices.csv")
print(data)

C:\kaggel\kaggel_tests\wfp_market_food_prices.csv


In [3]:
#Loading Data
gfp = pd.read_csv(data, engine='python')
gfp.head() # historico do preço p localidade p o produto durante 5 meses

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,pt_id,pt_name,um_id,um_name,mp_month,mp_year,mp_price,mp_commoditysource
0,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,1,2014,50.0000,WFP
1,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,2,2014,50.0000,WFP
2,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,3,2014,50.0000,WFP
3,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,4,2014,50.0000,WFP
4,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,5,2014,50.0000,WFP
5,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,6,2014,50.0000,WFP
6,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,7,2014,50.0000,WFP
7,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,8,2014,50.0000,WFP
8,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,9,2014,50.0000,WFP
9,1,Afghanistan,272,Badakhshan,266,Fayzabad,55,Bread,87,AFN,15,Retail,5,KG,10,2014,50.0000,WFP


cm_id = (comodity_purchase) - id do produto


In [4]:
gfp.shape

(743914, 18)

### Data Treating

In [5]:
#rename«ing cols
gfp.columns = ['country_id', 'country_name', 'locality_id', 'locality_name', 'market_id', 'market_name', 'commodity_purchase_id', 'commodity_purchased', 'currency_id', 'name_of_currency', 'market_type_id', 'market_type', 'measurement_id', 'unit_of_goods_measurement', 'month_recorded', 'year_recorded','mp_price:_price_paid', 'source_supplying_price_information']
gfp.columns = [x.upper() for x in gfp.columns]

In [6]:
#creating a list only with id cols
id_final = [col for col in gfp.columns if col.endswith('_ID')]

In [7]:
#droping id's col since doesnt add nothing to the analysis
gfp.drop(id_final, axis=1, inplace=True)

In [8]:
# create dataframe without year, month and price, to obtain one entry for each different shop
gfp_unique_shop = gfp[['COUNTRY_NAME','LOCALITY_NAME','MARKET_NAME']]
gfp_unique_shop.head()
initialSize = gfp_unique_shop.shape[0]
print("initial dataset size", initialSize)
gfp_unique_shop.drop_duplicates(inplace=True) # cair as colunas para poder analisar mercado único por país (mercados totais por pais)
gfp_unique_shop.reset_index(inplace=True, drop=True) #após drop_duplicates always do a reset
finalSize = gfp_unique_shop.shape[0]
print("final dataset size", finalSize)

initial dataset size 743914
final dataset size 1478


In [9]:
# renaming col in order to not misunderstand
gfp_unique_shop.rename(columns={'MARKET_NAME':'MARKET_SHOPS'}, inplace=True) 

In [10]:
list_all_countries = gfp_unique_shop.groupby(by='COUNTRY_NAME')['COUNTRY_NAME'].nunique()
print( '[COUNTRIES] Total Contries in the df', len(list_all_countries))

[COUNTRIES] Total Contries in the df 74


In [11]:
list_all_markets = gfp_unique_shop.groupby(by='MARKET_SHOPS')['MARKET_SHOPS'].nunique()
print('[MARKETS] Total markets in the df', len(list_all_markets))

[MARKETS] Total markets in the df 1449


In [12]:
#total de lojas por país
gfp_unique_shop.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS
0,Afghanistan,Badakhshan,Fayzabad
1,Afghanistan,Balkh,Mazar
2,Afghanistan,Bamyan,Bamyan
3,Afghanistan,Faryab,Maymana
4,Afghanistan,Hirat,Hirat


In [13]:
#NR_TOTAL DE LOJAS Afghanistan # pesquisa - apagar
gfp_unique_shop.loc[gfp_unique_shop['COUNTRY_NAME'] == 'Afghanistan']

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS
0,Afghanistan,Badakhshan,Fayzabad
1,Afghanistan,Balkh,Mazar
2,Afghanistan,Bamyan,Bamyan
3,Afghanistan,Faryab,Maymana
4,Afghanistan,Hirat,Hirat
5,Afghanistan,Kabul,Kabul
6,Afghanistan,Kandahar,Kandahar
7,Afghanistan,Nangarhar,Jalalabad
8,Afghanistan,Paktya,Gardez
9,Afghanistan,$Daykundi,Nili


In [14]:
#TOTAL DE LOJAS EM QUE O NOME DA LOJA É IGUAL À LOCALIDADE

df_teste = gfp_unique_shop.loc[gfp_unique_shop['LOCALITY_NAME'] == gfp_unique_shop['MARKET_SHOPS']]

In [15]:
#df groupby object, para passar p df, utiliza-se a função: .apply(list)
df_teste_COUNTRY_2 = df_teste.groupby('COUNTRY_NAME').MARKET_SHOPS.apply(list)

In [16]:
df_teste_COUNTRY_2 = df_teste_COUNTRY_2.to_frame()

In [17]:
type(df_teste_COUNTRY_2)

pandas.core.frame.DataFrame

In [18]:
# listagem de todas as lojas diferentes em cada país
df_teste_COUNTRY_2['COUNT_SHOPS_C'] = df_teste_COUNTRY_2['MARKET_SHOPS'].str.len()

In [19]:
gfp.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_NAME,COMMODITY_PURCHASED,NAME_OF_CURRENCY,MARKET_TYPE,UNIT_OF_GOODS_MEASUREMENT,MONTH_RECORDED,YEAR_RECORDED,MP_PRICE:_PRICE_PAID,SOURCE_SUPPLYING_PRICE_INFORMATION
0,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,1,2014,50.0,WFP
1,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,2,2014,50.0,WFP
2,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,3,2014,50.0,WFP
3,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,4,2014,50.0,WFP
4,Afghanistan,Badakhshan,Fayzabad,Bread,AFN,Retail,KG,5,2014,50.0,WFP


In [20]:
#tipos de comercio distinto
df_types_shop = gfp.MARKET_TYPE.unique()

In [21]:
# type numpy.ndarray
print(df_types_shop)

['Retail' 'Wholesale' 'Producer' 'Farm Gate']


In [22]:
#passing numpy.ndarray to a df
df_unique_type_markets = pd.DataFrame(data=df_types_shop)    # values
#index=np.array(0,3))   # 1st column as index)

In [23]:
#inputing names in the cols
df_unique_type_markets = df_unique_type_markets.rename(columns = {df_unique_type_markets.columns[0]:'MARKET_TYPES'})
#, df_unique_type_markets.columns[1]:'MARKET_TYPES'

In [24]:
df_unique_type_markets.head()

,MARKET_TYPES
0,Retail
1,Wholesale
2,Producer
3,Farm Gate


In [47]:
#ir buscar as percentagens para cada mercado por país
df_mercados_pais = gfp_unique_shop.copy()
df_mercados_pais['MARKET_TYPE'] = gfp.MARKET_TYPE
df_mercados_pais = df_mercados_pais.drop_duplicates(keep='first') # retirar todas as linhas a mais devido ao histórico de compras
df_mercados_pais.head()

,COUNTRY_NAME,LOCALITY_NAME,MARKET_SHOPS,MARKET_TYPE
0,Afghanistan,Badakhshan,Fayzabad,Retail
1,Afghanistan,Balkh,Mazar,Retail
2,Afghanistan,Bamyan,Bamyan,Retail
3,Afghanistan,Faryab,Maymana,Retail
4,Afghanistan,Hirat,Hirat,Retail


In [48]:
#nr_total de mercados(cidade) diferentes por país
df_mercados_dif_pais = gfp_unique_shop.groupby('COUNTRY_NAME').size() # after a groupby reset_index?
df_mercados_dif_pais = df_mercados_dif_pais.reset_index() #do not drop the index in order to keep th countries
df_mercados_dif_pais.rename(columns={0:'COUNT_MARKET'}, inplace=True) 
df_mercados_dif_pais = df_mercados_dif_pais.sort_values(by=['COUNT_MARKET'], ascending=False) # after a groupby reset_index?
df_mercados_dif_pais = df_mercados_dif_pais.reset_index(drop=True) #do not drop the index in order to keep th countries
df_mercados_dif_pais.head()

,COUNTRY_NAME,COUNT_MARKET
0,Mali,103
1,Zambia,75
2,Niger,73
3,Malawi,72
4,Myanmar,66


In [ ]:
#nr total de comercio global
